# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 21 2023 11:36AM,259639,10,9220343-BO,Eywa Pharma Inc.,Released
1,Mar 21 2023 11:33AM,259638,10,MSP219756,"Methapharm, Inc.",Released
2,Mar 21 2023 11:33AM,259638,10,MSP219763,"Methapharm, Inc.",Released
3,Mar 21 2023 11:33AM,259638,10,MSP219764,"Methapharm, Inc.",Released
4,Mar 21 2023 11:33AM,259638,10,MSP219765,"Methapharm, Inc.",Released
5,Mar 21 2023 11:33AM,259638,10,MSP219766,"Methapharm, Inc.",Released
6,Mar 21 2023 11:33AM,259638,10,MSP219767,"Methapharm, Inc.",Released
7,Mar 21 2023 11:33AM,259638,10,MSP219770,"Methapharm, Inc.",Released
8,Mar 21 2023 11:33AM,259638,10,MSP219772,"Methapharm, Inc.",Released
9,Mar 21 2023 11:33AM,259638,10,MSP219777,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,259635,Released,9
26,259636,Released,7
27,259637,Released,1
28,259638,Released,9
29,259639,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259635,NaN,NaN,9.0
259636,NaN,NaN,7.0
259637,NaN,NaN,1.0
259638,NaN,NaN,9.0
259639,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259475,0.0,20.0,0.0
259574,0.0,1.0,0.0
259596,0.0,1.0,0.0
259599,0.0,2.0,0.0
259606,16.0,26.0,31.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259475,0,20,0
259574,0,1,0
259596,0,1,0
259599,0,2,0
259606,16,26,31


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259475,0,20,0
1,259574,0,1,0
2,259596,0,1,0
3,259599,0,2,0
4,259606,16,26,31


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259475,,20,
1,259574,,1,
2,259596,,1,
3,259599,,2,
4,259606,16,26,31


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 21 2023 11:36AM,259639,10,Eywa Pharma Inc.
1,Mar 21 2023 11:33AM,259638,10,"Methapharm, Inc."
10,Mar 21 2023 11:32AM,259637,10,"Citieffe, Inc."
11,Mar 21 2023 11:32AM,259636,10,"Digital Brands, LLC"
18,Mar 21 2023 11:30AM,259635,10,"Methapharm, Inc."
26,Mar 21 2023 11:30AM,259635,10,Methapharm-G
27,Mar 21 2023 11:16AM,259632,15,"Brookfield Pharmaceuticals, LLC"
52,Mar 21 2023 11:04AM,259630,10,Bio-PRF
57,Mar 21 2023 10:55AM,259629,19,"Emersa Waterbox, LLC"
58,Mar 21 2023 10:42AM,259628,18,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 21 2023 11:36AM,259639,10,Eywa Pharma Inc.,,,1
1,Mar 21 2023 11:33AM,259638,10,"Methapharm, Inc.",,,9
2,Mar 21 2023 11:32AM,259637,10,"Citieffe, Inc.",,,1
3,Mar 21 2023 11:32AM,259636,10,"Digital Brands, LLC",,,7
4,Mar 21 2023 11:30AM,259635,10,"Methapharm, Inc.",,,9
5,Mar 21 2023 11:30AM,259635,10,Methapharm-G,,,9
6,Mar 21 2023 11:16AM,259632,15,"Brookfield Pharmaceuticals, LLC",,1,24
7,Mar 21 2023 11:04AM,259630,10,Bio-PRF,,2,3
8,Mar 21 2023 10:55AM,259629,19,"Emersa Waterbox, LLC",,,1
9,Mar 21 2023 10:42AM,259628,18,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 11:36AM,259639,10,Eywa Pharma Inc.,1,,
1,Mar 21 2023 11:33AM,259638,10,"Methapharm, Inc.",9,,
2,Mar 21 2023 11:32AM,259637,10,"Citieffe, Inc.",1,,
3,Mar 21 2023 11:32AM,259636,10,"Digital Brands, LLC",7,,
4,Mar 21 2023 11:30AM,259635,10,"Methapharm, Inc.",9,,
5,Mar 21 2023 11:30AM,259635,10,Methapharm-G,9,,
6,Mar 21 2023 11:16AM,259632,15,"Brookfield Pharmaceuticals, LLC",24,1,
7,Mar 21 2023 11:04AM,259630,10,Bio-PRF,3,2,
8,Mar 21 2023 10:55AM,259629,19,"Emersa Waterbox, LLC",1,,
9,Mar 21 2023 10:42AM,259628,18,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 11:36AM,259639,10,Eywa Pharma Inc.,1,,
1,Mar 21 2023 11:33AM,259638,10,"Methapharm, Inc.",9,,
2,Mar 21 2023 11:32AM,259637,10,"Citieffe, Inc.",1,,
3,Mar 21 2023 11:32AM,259636,10,"Digital Brands, LLC",7,,
4,Mar 21 2023 11:30AM,259635,10,"Methapharm, Inc.",9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 11:36AM,259639,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Mar 21 2023 11:33AM,259638,10,"Methapharm, Inc.",9.0,NaN,NaN
2,Mar 21 2023 11:32AM,259637,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Mar 21 2023 11:32AM,259636,10,"Digital Brands, LLC",7.0,NaN,NaN
4,Mar 21 2023 11:30AM,259635,10,"Methapharm, Inc.",9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 21 2023 11:36AM,259639,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Mar 21 2023 11:33AM,259638,10,"Methapharm, Inc.",9.0,0.0,0.0
2,Mar 21 2023 11:32AM,259637,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Mar 21 2023 11:32AM,259636,10,"Digital Brands, LLC",7.0,0.0,0.0
4,Mar 21 2023 11:30AM,259635,10,"Methapharm, Inc.",9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2855843,41.0,5.0,0.0
15,519107,24.0,21.0,0.0
18,1817321,7.0,0.0,0.0
19,1038460,33.0,28.0,16.0
20,259623,10.0,22.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2855843,41.0,5.0,0.0
1,15,519107,24.0,21.0,0.0
2,18,1817321,7.0,0.0,0.0
3,19,1038460,33.0,28.0,16.0
4,20,259623,10.0,22.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,41.0,5.0,0.0
1,15,24.0,21.0,0.0
2,18,7.0,0.0,0.0
3,19,33.0,28.0,16.0
4,20,10.0,22.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,41.0
1,15,Released,24.0
2,18,Released,7.0
3,19,Released,33.0
4,20,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20
Status,,,,,
Completed,0.0,0.0,0.0,16.0,0.0
Executing,5.0,21.0,0.0,28.0,22.0
Released,41.0,24.0,7.0,33.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20
0,Completed,0.0,0.0,0.0,16.0,0.0
1,Executing,5.0,21.0,0.0,28.0,22.0
2,Released,41.0,24.0,7.0,33.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20
0,Completed,0.0,0.0,0.0,16.0,0.0
1,Executing,5.0,21.0,0.0,28.0,22.0
2,Released,41.0,24.0,7.0,33.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()